### **Loading Required Libraries**

In [ ]:
import torch
from datasets import load_dataset
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token has not been saved to git credential helper.


In [ ]:
from transformers import __version__ as transformers_version
print(transformers_version)

4.42.2


# **Finetuning Qwen2.5-7B Model on Salesforce/Xlam-function-calling Dataset**

### **Loading the Base Model**

In [ ]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,        # Enable 8-bit loading
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Base Model Inference for JSON Generation**

In [ ]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = 'Convert the following invalid json into valid json in single line: {"employee": "name": "Jane Smith", "designation": "Software Engineer"}'
messages = [
    {"role": "system", "content": "You are Qwen, a helpful assistant. Your task is to convert any input into valid json"},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'{"employee": {"name": "Jane Smith", "designation": "Software Engineer"}}'

In [ ]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

### **Loading the Dataset**

In [ ]:
dataset = load_dataset("Salesforce/xlam-function-calling-60k",split="train")
dataset

Dataset({
    features: ['answers', 'query', 'tools', 'id'],
    num_rows: 60000
})

In [ ]:
dataset["query"][0]

'Where can I find live giveaways for beta access and games?'

In [ ]:
dataset["answers"][0]

'[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]'

### **Formatting the Dataset**

In [ ]:
def generate_query_answer_prompt(data_point):
    """Generate a prompt for fine-tuning based on the query and answers from the dataset."""

    messages = []

    # Add the system prompt with the specific instruction
    messages.append({
        'role': 'system',
        'content': 'You are a helpful assistant. Your task is to convert every input query into a valid JSON.'
    })

    # Add the user's query
    messages.append({
        'role': 'user',
        'content': data_point['query']
    })

    # Add the assistant's response
    messages.append({
        'role': 'assistant',
        'content': data_point['answers']
    })

    prompt = ''

    # Since we are not using tools, we follow the 'else' branch of the template
    # Handle the system message
    if messages[0]['role'] == 'system':
        prompt += '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n'
    else:
        prompt += '<|im_start|>system\nYou are a helpful assistant. Your task is to convert every input query into a valid JSON.<|im_end|>\n'

    # Process the rest of the messages
    for message in messages[1:]:
        role = message['role']
        content = message['content']

        # Since we're not using tool_calls or tools, we can format messages directly
        prompt += f'<|im_start|>{role}\n{content}<|im_end|>\n'

    return prompt

# Apply the function to the dataset to create the 'prompt' column
text_column = [generate_query_answer_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [ ]:
dataset

Dataset({
    features: ['answers', 'query', 'tools', 'id', 'prompt'],
    num_rows: 60000
})

### **Tokenization of Dataset**

In [ ]:
dataset = dataset.shuffle(seed=1234)
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset

Dataset({
    features: ['answers', 'query', 'tools', 'id', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 60000
})

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]
dataset

DatasetDict({
    train: Dataset({
        features: ['answers', 'query', 'tools', 'id', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 48000
    })
    test: Dataset({
        features: ['answers', 'query', 'tools', 'id', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 12000
    })
})

In [ ]:
train_data['prompt'][0]

'<|im_start|>system\nYou are a helpful assistant. Your task is to convert every input query into a valid JSON.<|im_end|>\n<|im_start|>user\nCan you fetch the options prices for a Call option on AAPL with a strike price of 130 and an expiration date of 2023-12-31?<|im_end|>\n<|im_start|>assistant\n[{"name": "options_prices", "arguments": {"type": "Call", "symbol": "AAPL", "strike": 130, "expiration": "2023-12-31"}}]<|im_end|>\n'

### **Apply Lora**

In [ ]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],  # Target LoRA modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" 
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | Total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 161480704 | Total: 7777097216 | Percentage: 2.0764%


### **Setting Up Training Arguments and Training for SFTTrainer**

In [ ]:
import transformers
from trl import SFTTrainer

torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=50,
        warmup_steps=10,
        max_steps=1500,
        learning_rate=2e-4,
        logging_steps=5,
        output_dir="outputs_qwen2.5_7b_json",
        optim="adamw_hf",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# Run training
trainer.train()

/home/geek/anaconda3/envs/gemma2/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/geek/anaconda3/envs/gemma2/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/geek/anaconda3/envs/gemma2/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
max_steps is given, it will override any 

[2024-10-28 14:18:34,440] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/geek/anaconda3/envs/gemma2/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/geek/anaconda3/envs/gemma2/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/geek/anaconda3/envs/gemma2/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


  0%|          | 0/1500 [00:00<?, ?it/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/home/geek/anaconda3/envs/gemma2/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'loss': 2.8936, 'grad_norm': 1.321012020111084, 'learning_rate': 0.0001, 'epoch': 0.0}
{'loss': 1.7582, 'grad_norm': 1.2027846574783325, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 1.042, 'grad_norm': 0.619357705116272, 'learning_rate': 0.00019932885906040267, 'epoch': 0.0}
{'loss': 0.726, 'grad_norm': 0.3536224067211151, 'learning_rate': 0.0001986577181208054, 'epoch': 0.0}
{'loss': 0.718, 'grad_norm': 0.4736764430999756, 'learning_rate': 0.00019798657718120806, 'epoch': 0.0}
{'loss': 0.7451, 'grad_norm': 0.3898603916168213, 'learning_rate': 0.00019731543624161075, 'epoch': 0.0}
{'loss': 0.7024, 'grad_norm': 0.4720907509326935, 'learning_rate': 0.00019664429530201342, 'epoch': 0.0}
{'loss': 0.7805, 'grad_norm': 0.3852503001689911, 'learning_rate': 0.00019597315436241613, 'epoch': 0.0}
{'loss': 0.7006, 'grad_norm': 0.3920633792877197, 'learning_rate': 0.0001953020134228188, 'epoch': 0.0}
{'loss': 0.7133, 'grad_norm': 0.31026753783226013, 'learning_rate': 0.0001946308724832215, 'ep

TrainOutput(global_step=1500, training_loss=0.5824175117015838, metrics={'train_runtime': 2339.5141, 'train_samples_per_second': 2.565, 'train_steps_per_second': 0.641, 'total_flos': 3.021170654820864e+16, 'train_loss': 0.5824175117015838, 'epoch': 0.125})

### **Merge and Save the Finetuned Model**

In [ ]:
new_model = "qwen-2.5-7b-finetune-xlam-function-calling-60k"
trainer.model.save_pretrained(new_model)
model_id = "Qwen/Qwen2.5-7B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    load_in_8bit=True,        # Enable 8-bit loading
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

merged_model.save_pretrained("merged_model_sft2.5_7b_json",safe_serialization=True)
tokenizer.save_pretrained("merged_model_sft2.5_7b_json")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/geek/anaconda3/envs/gemma2/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:71: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


[2024-10-28 15:06:08,962] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/geek/anaconda3/envs/gemma2/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/geek/anaconda3/envs/gemma2/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


### **Loading the Saved Model**

In [ ]:
finetuned_model = AutoModelForCausalLM.from_pretrained("merged_model_sft2.5_7b_json", torch_dtype=torch.float16, device_map={"": 0})

finetuned_tokenizer = AutoTokenizer.from_pretrained("merged_model_sft2.5_7b_json")
finetuned_tokenizer.pad_token = finetuned_tokenizer.eos_token
finetuned_tokenizer.padding_side = "left"

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Push the model to Huggingface**

In [ ]:
repo_name = "Amin-01/qwen-2.5-7b-finetuned-for-json-generation-bnb-8bit"

finetuned_model.push_to_hub(repo_name)
finetuned_tokenizer.push_to_hub(repo_name)

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abdulmannan-01/qwen-2.5-7b-finetuned-for-json-generation-bnb-8bit/commit/f6c0ba0d5818607bfc5165599b74966bfbe02021', commit_message='Upload tokenizer', commit_description='', oid='f6c0ba0d5818607bfc5165599b74966bfbe02021', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abdulmannan-01/qwen-2.5-7b-finetuned-for-json-generation-bnb-8bit', endpoint='https://huggingface.co', repo_type='model', repo_id='abdulmannan-01/qwen-2.5-7b-finetuned-for-json-generation-bnb-8bit'), pr_revision=None, pr_num=None)

# **Inference Using Finetuned Huggingface Model**

### **Load Model From HuggingFace**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

finetuned_model = AutoModelForCausalLM.from_pretrained(
    "Amin-01/qwen-2.5-7b-finetuned-for-json-generation-bnb-8bit",
    torch_dtype=torch.float16,
    device_map="auto"
)
finetuned_tokenizer = AutoTokenizer.from_pretrained("Amin-01/qwen-2.5-7b-finetuned-for-json-generation-bnb-8bit")
finetuned_tokenizer.pad_token = finetuned_tokenizer.eos_token
finetuned_tokenizer.padding_side = 'left'

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors.index.json:   0%|          | 0.00/43.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def prepare_input(query):
    prompt = '<|im_start|>system\nYou are a helpful assistant. Your task is to convert every input query into a valid JSON.<|im_end|>\n'
    prompt += f'<|im_start|>user\n{query}<|im_end|>\n'
    prompt += '<|im_start|>assistant\n'

    # Tokenize the prompt
    inputs = finetuned_tokenizer(prompt, return_tensors='pt').to("cuda")
    return inputs

In [ ]:
def extract_assistant_response(output_text):
    # Extract content between <|im_start|>assistant and <|im_end|>
    assistant_start = '<|im_start|>assistant\n'
    assistant_end = '<|im_end|>'

    start_idx = output_text.find(assistant_start)
    if start_idx == -1:
        print("Assistant start token not found in the output.")
        return ''
    start_idx += len(assistant_start)

    end_idx = output_text.find(assistant_end, start_idx)
    if end_idx == -1:
        print("Assistant end token not found in the output.")
        assistant_content = output_text[start_idx:]
    else:
        assistant_content = output_text[start_idx:end_idx]

    return assistant_content.strip()

In [ ]:
def generate_response(model, inputs):
    # Generate the output
    output = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        pad_token_id=finetuned_tokenizer.eos_token_id,
        eos_token_id=finetuned_tokenizer.eos_token_id
    )

    output_text = finetuned_tokenizer.decode(output[0], skip_special_tokens=False)
    #print('Model Output:', output_text)

    assistant_response = extract_assistant_response(output_text)

    return assistant_response

In [ ]:
import json

def isJsonString(input_string):
    try:
        json.loads(input_string)
        return True
    except ValueError:
        return False

### **Testing on Single Text Queries**

In [ ]:
query1 = "Fetch auto-complete results for 'winter coat' from Kohls and 'kitchen appliance' from Wayfair."
inputs1 = prepare_input(query1)
response1 = generate_response(finetuned_model, inputs1)

print("Generated JSON response:", response1)

Generated JSON response: [{"name": "auto_complete", "arguments": {"query": "winter coat"}}, {"name": "auto_complete", "arguments": {"query": "kitchen appliance"}}]


In [ ]:
isJsonString(response1)

True

### **Testing on Multiple Text Queries**

In [ ]:
queries = [
    # 5 big textual prompts
    "I need an analysis report on the impact of our latest marketing campaign. Include data such as conversion rates, customer engagement metrics, social media performance, and any sales growth. Provide insights on which segments of the campaign were most successful and suggest areas for improvement.",
    "Please generate a monthly sales report for our e-commerce platform, detailing total sales, top-performing products, customer demographics, and return rates. Include comparisons to the previous month and highlight any notable trends or anomalies.",
    "Create a technical specification document for the new software system we're developing. Include sections on system architecture, database design, user interfaces, security protocols, scalability requirements, and integration with existing systems.",
    "Can you provide a strategic plan for expanding our business into new international markets? The plan should cover potential target countries, market entry strategies, competitive analysis, legal considerations, and estimated costs. Highlight key opportunities and risks.",
    "Generate a risk assessment report for our cloud infrastructure. The report should analyze potential security threats, vulnerabilities, compliance issues, and disaster recovery plans. Recommend best practices for improving system security and resilience.",
    # 25 big and complex invalid JSONs
    "{'payment': {'transactionID': 456789, 'amount': 100.00, 'currency': 'USD', 'method': 'Credit Card', 'status': 'Pending', 'details': {'cardNumber': '**** **** **** 1234', 'expiryDate': '12/23'}}}",  # Valid JSON
    "{'portfolio': {'investor': 'Jane Doe', 'stocks': [{'ticker': 'AAPL', 'shares': 50, 'averagePrice': 150}, {'ticker': 'GOOG', 'shares': 30, 'averagePrice': 2500}], 'totalValue': 180000}",  # Missing closing brace
    "{'jobApplication': {'candidateName': 'Alice Doe', 'position': 'Software Engineer', 'status': 'Under Review', 'interviews': [{'round': 1, 'interviewer': 'John Smith'}, {'round': 2, 'interviewer': 'Sara Lee'}], 'notes': 'Strong candidate, needs to improve coding skills'}}",  # Valid JSON
    "{'orderID': 12345, 'customer': {'name': 'Alice Johnson', 'email': 'alice@example.com'}, 'items': [{'productID': 567, 'productName': 'Laptop', 'quantity': 1, 'price': 999.99}, {'productID': 789, 'productName': 'Mouse', 'quantity': 2, 'price': 19.99}], 'total': 1039.97",  # Missing closing brace
    "{'shipment': {'trackingID': 987654321, 'origin': 'Los Angeles', 'destination': 'New York', 'items': [{'itemID': 456, 'name': 'Smartphone', 'quantity': 50}, {'itemID': 789, 'name': 'Laptop', 'quantity': 20}], 'status': 'In Transit'",  # Missing closing brace
    "{'conferenceCall': {'callID': 456123, 'participants': ['John Doe', 'Jane Smith'], 'agenda': 'Project Planning', 'date': '2022-05-01', 'time': '10:00 AM', 'duration': '1h'}}",  # Valid JSON
    "{'patient': {'name': 'Alice Doe', 'age': 30, 'medicalHistory': [{'condition': 'Diabetes', 'diagnosed': '2015'}, {'condition': 'Hypertension', 'diagnosed': '2019'}], 'medications': [{'name': 'Metformin', 'dosage': '500mg'}, {'name': 'Lisinopril', 'dosage': '10mg'}]}",  # Missing closing brace
    "{'movie': {'title': 'Inception', 'director': 'Christopher Nolan', 'year': 2010, 'genres': ['Sci-Fi', 'Thriller'], 'cast': [{'name': 'Leonardo DiCaprio', 'role': 'Dom Cobb'}, {'name': 'Joseph Gordon-Levitt', 'role': 'Arthur'}], 'ratings': {'IMDB': 8.8, 'Rotten Tomatoes': '87%'}}}",  # Valid JSON
    "{'trip': {'destination': 'Tokyo', 'startDate': '2023-04-15', 'endDate': '2023-04-30', 'travelers': [{'name': 'Alice Johnson', 'passport': '123456789'}, {'name': 'Bob Williams', 'passport': '987654321'}], 'itinerary': [{'day': 1, 'activity': 'Visit Tokyo Tower'}, {'day': 2, 'activity': 'Explore Shibuya Crossing'}], 'hotel': 'The Ritz-Carlton', 'roomType': 'Suite'}}",  # Valid JSON
    "{'ecommerce': {'orderID': 987654321, 'customer': {'name': 'Emily Smith', 'email': 'emily@example.com'}, 'items': [{'productID': 12345, 'name': 'Smart TV', 'quantity': 1, 'price': 499.99}, {'productID': 67890, 'name': 'Bluetooth Speaker', 'quantity': 2, 'price': 29.99}], 'total': 559.97, 'status': 'Shipped', 'shipping': {'address': '789 Birch Street', 'city': 'Austin', 'state': 'TX', 'zipcode': 73301'}}}",  # Valid JSON
    "{'fitness': {'workoutID': 101, 'type': 'Strength Training', 'exercises': [{'name': 'Bench Press', 'sets': 3, 'reps': 10, 'weight': 135}, {'name': 'Squats', 'sets': 4, 'reps': 12, 'weight': 185}], 'date': '2023-07-10', 'duration': '1 hour'}}",  # Missing closing brace
    "{'concert': {'name': 'Rock Fest 2023', 'location': 'Los Angeles', 'date': '2023-09-20', 'lineup': [{'band': 'The Rolling Stones', 'time': '8:00 PM'}, {'band': 'Queen', 'time': '9:30 PM'}], 'tickets': [{'section': 'VIP', 'price': 500}, {'section': 'General Admission', 'price': 150}], 'available': True}",  # Missing closing brace
    "{'conference': {'name': 'AI Summit', 'date': '2022-10-15', 'location': 'New York', 'topics': ['AI', 'Machine Learning', 'Deep Learning'], 'speakers': [{'name': 'John Smith', 'topic': 'AI'}, {'name': 'Emily Johnson', 'topic': 'Machine Learning'}]",  # Missing closing brace
    "{'conference': {'name': 'Tech Expo 2023', 'location': 'San Francisco', 'date': '2023-10-15', 'topics': ['AI', 'Blockchain', 'Cybersecurity'], 'speakers': [{'name': 'John McAfee', 'topic': 'Cybersecurity'}, {'name': 'Vitalik Buterin', 'topic': 'Blockchain'}], 'attendees': 3000}}",  # Valid JSON
    "{'survey': {'title': 'Customer Satisfaction', 'responses': [{'question': 'How would you rate our service?', 'rating': 5}, {'question': 'What can we improve?', 'text': 'Faster shipping'}], 'summary': {'averageRating': 4.8, 'commonSuggestions': ['Lower prices', 'More products']}}}",  # Valid JSON
    "{'socialMedia': {'platform': 'Twitter', 'user': {'username': 'janedoe', 'followers': 5000, 'following': 300, 'tweets': [{'date': '2023-07-10', 'content': 'Just got a new job!'}, {'date': '2023-08-01', 'content': 'Loving my new apartment.'}], 'likes': 100, 'retweets': 10}}}",  # Missing closing brace
    "{'artExhibit': {'name': 'Modern Art Showcase', 'location': 'Museum of Fine Arts', 'date': '2023-11-05', 'artists': [{'name': 'Pablo Picasso', 'works': ['The Weeping Woman', 'Guernica']}, {'name': 'Vincent van Gogh', 'works': ['Starry Night', 'Sunflowers']}], 'tickets': {'price': 25, 'available': True}}}",  # Missing closing brace
    "{'banking': {'accountHolder': 'John Doe', 'accountNumber': '12345678', 'transactions': [{'date': '2023-01-01', 'description': 'Deposit', 'amount': 1000}, {'date': '2023-02-15', 'description': 'Withdrawal', 'amount': 500}], 'balance': 500}}",  # Valid JSON
    "{'scientificResearch': {'studyID': 654321, 'title': 'Gene Therapy for Cancer', 'researchers': [{'name': 'Dr. Alice Doe', 'institution': 'Harvard University'}, {'name': 'Dr. Bob Smith', 'institution': 'MIT'}], 'status': 'In Progress', 'funding  5000000}}",
    "{'musicConcert': {'concertID': 987123, 'artist': 'The Beatles', 'date': '1965-08-15', 'venue': 'Shea Stadium', 'setlist': [{'song': 'Twist and Shout'}, {'song': 'Help!'}], 'tickets': {'price': 5, 'soldOut': True}}}",  # Valid JSON
    "{'fitnessChallenge': {'challengeID': 456789, 'name': '30-Day Fitness Challenge', 'participants': [{'name': 'John Doe', 'progress': 'Week 2'}, {'name': 'Jane Smith', 'progress': 'Week 3'}], 'startDate': '2023-01-01', 'endDate' '2023-01-30'}",
    "{'stockMarket': {'stock': {'symbol': 'AAPL', 'company': 'Apple Inc.', 'price': 150.50, 'marketCap': '2.5T', 'dividendYield': '0.5%'}, 'sector': 'Technology'}}",  # Missing closing brace
    "{'movieReview': {'title': 'The Godfather', 'director': 'Francis Ford Coppola', 'year': 1972, 'ratings': {'IMDB': 9.2, 'Rotten Tomatoes': '98%'}, 'review': 'An absolute classic. A must-watch for any film enthusiast.'}",
    "{'onlineCourse': {'courseID': 123456, 'title': 'Data Science 101', 'instructor': {'name': 'Dr. Alice Doe', 'institution': 'MIT'}, 'students': 500, 'startDate': '2023-09-01', 'endDate': '2023-12-01'}",
    "{'travelItinerary': {'itineraryID': 789012, 'destination': 'Paris', 'departureDate': '2023-06-15', 'returnDate': '2023-06-30', 'traveler': {'name': 'John Doe', 'passport': '123456789'}, 'accommodation': {'hotel': 'Le Meurice', 'roomType': 'Deluxe Suite'}}}",  # Valid JSON
]

**Evaluation Function**

In [ ]:
def evaluate_model_performance(model, queries, tokenizer, max_retries=5):
    total_queries = len(queries)
    valid_json_count = 0

    for i, query in enumerate(queries):
        inputs = prepare_input(query)
        response = None

        # Retry mechanism for invalid JSONs
        for attempt in range(max_retries):
            response = generate_response(model, inputs)
            if isJsonString(response):
                break

        is_valid = isJsonString(response)

        print(f"Query {i+1}: {query}")
        print(f"Generated JSON response: {response}")

        if is_valid:
            print("The response is valid JSON.")
            valid_json_count += 1
        else:
            print("The response is not valid JSON.")
        print("-" * 50)

    # Compute the percentage of valid JSON outputs
    valid_percentage = (valid_json_count / total_queries) * 100
    print(f"Valid JSON responses: {valid_json_count}/{total_queries} ({valid_percentage:.2f}%)")


In [ ]:
evaluate_model_performance(finetuned_model, queries, finetuned_tokenizer)

Query 1: I need an analysis report on the impact of our latest marketing campaign. Include data such as conversion rates, customer engagement metrics, social media performance, and any sales growth. Provide insights on which segments of the campaign were most successful and suggest areas for improvement.
Generated JSON response: [{"name": "marketing_campaign_analysis", "arguments": {"campaign_data": [{"conversion_rate": 0.15, "customer_engagement": 85, "social_media_performance": 75, "sales_growth": 20}, {"conversion_rate": 0.1, "customer_engagement": 70, "social_media_performance": 65, "sales_growth": 15}, {"conversion_rate": 0.2, "customer_engagement": 90, "social_media_performance": 80, "sales_growth": 30}]}}]
The response is valid JSON.
--------------------------------------------------
Query 2: Please generate a monthly sales report for our e-commerce platform, detailing total sales, top-performing products, customer demographics, and return rates. Include comparisons to the previ

## **QWen2.5-7B Base Model for JSON Mode**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

# Load the Qwen2.5-3B-Instruct model and tokenizer
model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    load_in_8bit=True,        # Enable 8-bit loading
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Function to extract JSON content between code block markers
def extract_json_content(response):
    if "```json" in response and "```" in response:
        start = response.find("```json") + len("```json")
        end = response.find("```", start)
        if end != -1:
            return response[start:end].strip()
    elif "```" in response:
        # If there's a code block without language specified
        start = response.find("```") + len("```")
        end = response.find("```", start)
        if end != -1:
            return response[start:end].strip()
    return response.strip()

In [ ]:
# Function to evaluate the model on a list of queries
def evaluate_model_performance(model, queries, tokenizer):
    valid_json_count = 0
    total_queries = len(queries)

    for i, query in enumerate(queries, start=1):
        print(f"Query {i}: {query}")

        # Prepare the prompt
        prompt = (
            "You are a helpful assistant. Your task is to convert any input into valid JSON.\n"
            f"Convert the following text query or invalid JSON into valid JSON."
            f"Only provide the JSON response, no extra information:\n{query}"
        )

        messages = [
            {
                "role": "system",
                "content": "You are Qwen, a helpful assistant. Your task is to convert any input into valid JSON."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]

        # Apply the chat template
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        # Generate the response
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7
        )
        # Remove the prompt tokens from the output
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        # Decode the generated tokens
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(f"Generated JSON response: {response}")

        # Extract JSON content if wrapped in code block
        json_content = extract_json_content(response)

        # Check if the extracted content is valid JSON
        is_valid = isJsonString(json_content)
        if is_valid:
            print("The response is valid JSON.")
            valid_json_count += 1
        else:
            print("The response is not valid JSON.")
        print("-" * 50)

    # Compute the percentage of valid JSON outputs
    valid_percentage = (valid_json_count / total_queries) * 100
    print(f"Valid JSON responses: {valid_json_count}/{total_queries} ({valid_percentage:.2f}%)")


In [ ]:
# Call the evaluation function
evaluate_model_performance(model, queries, tokenizer)

Query 1: I need an analysis report on the impact of our latest marketing campaign. Include data such as conversion rates, customer engagement metrics, social media performance, and any sales growth. Provide insights on which segments of the campaign were most successful and suggest areas for improvement.
Generated JSON response: ```json
{
  "request": {
    "type": "analysis_report",
    "topics": [
      "conversion_rates",
      "customer_engagement_metrics",
      "social_media_performance",
      "sales_growth"
    ],
    "segments": [
      "most_successful",
      "areas_for_improvement"
    ]
  }
}
```
The response is valid JSON.
--------------------------------------------------
Query 2: Please generate a monthly sales report for our e-commerce platform, detailing total sales, top-performing products, customer demographics, and return rates. Include comparisons to the previous month and highlight any notable trends or anomalies.
Generated JSON response: ```json
{
  "request": {

## **QWen2.5-7B Finetuned and Base Model Comparison**

### **Text Queries**

| **Query** | **Input Query** | **Base Model JSON** | **Base Model JSON Validity** | **Fine-tuned Model JSON** | **Fine-tuned Model JSON Validity** |
|---|---|---|---|---|---|
| **Query 1** | I need an analysis report on the impact of our latest marketing campaign. Include data such as conversion rates, customer engagement metrics, social media performance, and any sales growth. Provide insights on which segments of the campaign were most successful and suggest areas for improvement. | ` {"request": {"type": "analysis_report", "topics": ["conversion_rates", "customer_engagement_metrics", "social_media_performance", "sales_growth"], "segments": ["most_successful", "areas_for_improvement"]}}` | Valid JSON | ` [{"name": "marketing_campaign_analysis", "arguments": {"campaign_data": [{"conversion_rate": 0.15, "customer_engagement": 85, "social_media_performance": 75, "sales_growth": 20}]}}]` | Valid JSON |
| **Query 2** | Please generate a monthly sales report for our e-commerce platform, detailing total sales, top-performing products, customer demographics, and return rates. Include comparisons to the previous month and highlight any notable trends or anomalies. | ` {"request": {"type": "monthly_sales_report", "details": ["total_sales", "top_performing_products", "customer_demographics", "return_rates"], "platform": "e-commerce", "comparisons": true, "highlight_trends_anomalies": true}}` | Valid JSON | ` [{"name": "monthly_sales_report", "arguments": {"ecommerce_platform": true}}]` | Valid JSON |
| **Query 3** | Create a technical specification document for the new software system we're developing. Include sections on system architecture, database design, user interfaces, security protocols, scalability requirements, and integration with existing systems. | ` {"document": {"title": "Technical Specification Document", "sections": [{"section": "System Architecture", "description": "Detailed description."}, {"section": "Database Design", "description": "Database schema and design."}, {"section": "User Interfaces", "description": "Details about UI."}, {"section": "Security Protocols", "description": "Security measures."}, {"section": "Scalability Requirements", "description": "System scalability."}, {"section": "Integration with Existing Systems", "description": "Integration details."}]}}` | Valid JSON | ` [{"name": "technical_specification", "arguments": {"sections": ["system_architecture", "database_design", "user_interfaces", "security_protocols", "scalability_requirements", "integration_with_existing_systems"]}}]` | Valid JSON |
| **Query 4** | Can you provide a strategic plan for expanding our business into new international markets? The plan should cover potential target countries, market entry strategies, competitive analysis, legal considerations, and estimated costs. Highlight key opportunities and risks. | ` {"query": "Can you provide a strategic plan for expanding...", "requirements": {"target_countries": [], "market_entry_strategies": [], "competitive_analysis": {}, "legal_considerations": {}, "estimated_costs": {}, "opportunities": [], "risks": []}}` | Valid JSON | ` [{"name": "business_expansion_plan", "arguments": {"market": "international", "strategy": "entry_strategies", "analysis": "competitive_analysis", "legal": "legal_considerations", "costs": "estimated_costs"}}]` | Valid JSON |
| **Query 5** | Generate a risk assessment report for our cloud infrastructure. The report should analyze potential security threats, vulnerabilities, compliance issues, and disaster recovery plans. Recommend best practices for improving system security and resilience. | ` {"request": {"type": "risk_assessment_report", "scope": "cloud_infrastructure", "sections": [{"title": "Security Threats Analysis", "content": "Analyze potential security threats."}, {"title": "Vulnerabilities Assessment", "content": "Assess current vulnerabilities."}, {"title": "Compliance Issues Review", "content": "Evaluate compliance."}, {"title": "Disaster Recovery Plan Evaluation", "content": "Assess disaster recovery plan."}], "recommendations": {"improving_system_security": "Best practices.", "resilience_strategies": "Resilience strategies."}}` | Valid JSON | ` [{"name": "risk_assessment", "arguments": {"target": "cloud infrastructure", "scope": "security threats, vulnerabilities, compliance issues, disaster recovery plans"}}]` | Valid JSON |


### **Valid and Invalid Json Queries**

| **Query** | **Input Query** | **Base Model JSON** | **Base Model JSON Validity** | **Fine-tuned Model JSON** | **Fine-tuned Model JSON Validity** |
|---|---|---|---|---|---|
| **Query 6** | {'payment': {'transactionID': 456789, 'amount': 100.00, 'currency': 'USD', 'method': 'Credit Card', 'status': 'Pending', 'details': {'cardNumber': '**** **** **** 1234', 'expiryDate': '12/23'}}} | ` {"payment": {"transactionID": 456789, "amount": 100.00, "currency": "USD", "method": "Credit Card", "status": "Pending", "details": {"cardNumber": "**** **** **** 1234", "expiryDate": "12/23"}}}` | Valid JSON | ` [{"name": "verify_payment", "arguments": {"transaction_id": 456789, "amount": 100.0, "currency": "USD", "method": "Credit Card", "status": "Pending", "card_number": "**** **** **** 1234", "expiry_date": "12/23"}}]` | Valid JSON |
| **Query 7** | {'portfolio': {'investor': 'Jane Doe', 'stocks': [{'ticker': 'AAPL', 'shares': 50, 'averagePrice': 150}, {'ticker': 'GOOG', 'shares': 30, 'averagePrice': 2500}], 'totalValue': 180000} | ` {"portfolio": {"investor": "Jane Doe", "stocks": [{"ticker": "AAPL", "shares": 50, "averagePrice": 150}, {"ticker": "GOOG", "shares": 30, "averagePrice": 2500}], "totalValue": 180000}}` | Valid JSON | ` [{"name": "calculate_investment_return", "arguments": {"portfolio": {"investor": "Jane Doe", "stocks": [{"ticker": "AAPL", "shares": 50, "averagePrice": 150}, {"ticker": "GOOG", "shares": 30, "averagePrice": 2500}], "totalValue": 180000}}}]` | Valid JSON |
| **Query 8** | {'jobApplication': {'candidateName': 'Alice Doe', 'position': 'Software Engineer', 'status': 'Under Review', 'interviews': [{'round': 1, 'interviewer': 'John Smith'}, {'round': 2, 'interviewer': 'Sara Lee'}], 'notes': 'Strong candidate, needs to improve coding skills'}} | ` {"jobApplication": {"candidateName": "Alice Doe", "position": "Software Engineer", "status": "Under Review", "interviews": [{"round": 1, "interviewer": "John Smith"}, {"round": 2, "interviewer": "Sara Lee"}], "notes": "Strong candidate, needs to improve coding skills"}}` | Valid JSON | ` [{"name": "store_job_application", "arguments": {"candidate_name": "Alice Doe", "position": "Software Engineer", "status": "Under Review", "interviews": [{"round": 1, "interviewer": "John Smith"}, {"round": 2, "interviewer": "Sara Lee"}], "notes": "Strong candidate, needs to improve coding skills"}}]` | Valid JSON |
| **Query 9** | {'orderID': 12345, 'customer': {'name': 'Alice Johnson', 'email': 'alice@example.com'}, 'items': [{'productID': 567, 'productName': 'Laptop', 'quantity': 1, 'price': 999.99}, {'productID': 789, 'productName': 'Mouse', 'quantity': 2, 'price': 19.99}], 'total': 1039.97 | ` {"orderID": 12345, "customer": {"name": "Alice Johnson", "email": "alice@example.com"}, "items": [{"productID": 567, "productName": "Laptop", "quantity": 1, "price": 999.99}, {"productID": 789, "productName": "Mouse", "quantity": 2, "price": 19.99}], "total": 1039.97}` | Valid JSON | ` [{"name": "process_order", "arguments": {"order_id": 12345, "customer": {"name": "Alice Johnson", "email": "alice@example.com"}, "items": [{"product_id": 567, "product_name": "Laptop", "quantity": 1, "price": 999.99}, {"product_id": 789, "product_name": "Mouse", "quantity": 2, "price": 19.99}], "total": 1039.97}}]`| Valid JSON |
| **Query 10** | {'shipment': {'trackingID': 987654321, 'origin': 'Los Angeles', 'destination': 'New York', 'items': [{'itemID': 456, 'name': 'Smartphone', 'quantity': 50}, {'itemID': 789, 'name': 'Laptop', 'quantity': 20}], 'status': 'In Transit' | ` {"shipment": {"trackingID": 987654321, "origin": "Los Angeles", "destination": "New York", "items": [{"itemID": 456, "name": "Smartphone", "quantity": 50}, {"itemID": 789, "name": "Laptop", "quantity": 20}], "status": "In Transit"}}` | Valid JSON | ` [{"name": "track_shipment", "arguments": {"tracking_id": 987654321}}]` | Valid JSON |
| **Query 11** | {'conferenceCall': {'callID': 456123, 'participants': ['John Doe', 'Jane Smith'], 'agenda': 'Project Planning', 'date': '2022-05-01', 'time': '10:00 AM', 'duration': '1h'}} | ` {"conferenceCall": {"callID": 456123, "participants": ["John Doe", "Jane Smith"], "agenda": "Project Planning", "date": "2022-05-01", "time": "10:00 AM", "duration": "1h"}}` | Valid JSON | ` [{"name": "conference_call", "arguments": {"call_id": 456123, "participants": ["John Doe", "Jane Smith"], "agenda": "Project Planning", "date": "2022-05-01", "time": "10:00 AM", "duration": "1h"}}]` | Valid JSON |
| **Query 12** | {'patient': {'name': 'Alice Doe', 'age': 30, 'medicalHistory': [{'condition': 'Diabetes', 'diagnosed': '2015'}, {'condition': 'Hypertension', 'diagnosed': '2019'}], 'medications': [{'name': 'Metformin', 'dosage': '500mg'}, {'name': 'Lisinopril', 'dosage': '10mg'}]} | ` {"patient": {"name": "Alice Doe", "age": 30, "medicalHistory": [{"condition": "Diabetes", "diagnosed": "2015"}, {"condition": "Hypertension", "diagnosed": "2019"}], "medications": [{"name": "Metformin", "dosage": "500mg"}, {"name": "Lisinopril", "dosage": "10mg"}]}}` | Valid JSON | ` [{"name": "review_medical_history", "arguments": {"medical_history": [{"condition": "Diabetes", "diagnosed": "2015"}, {"condition": "Hypertension", "diagnosed": "2019"}]}}]` | Valid JSON |
| **Query 13** | {'movie': {'title': 'Inception', 'director': 'Christopher Nolan', 'year': 2010, 'genres': ['Sci-Fi', 'Thriller'], 'cast': [{'name': 'Leonardo DiCaprio', 'role': 'Dom Cobb'}, {'name': 'Joseph Gordon-Levitt', 'role': 'Arthur'}], 'ratings': {'IMDB': 8.8, 'Rotten Tomatoes': '87%'}}} | ` {"movie": {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "genres": ["Sci-Fi", "Thriller"], "cast": [{"name": "Leonardo DiCaprio", "role": "Dom Cobb"}, {"name": "Joseph Gordon-Levitt", "role": "Arthur"}], "ratings": {"IMDB": 8.8, "Rotten Tomatoes": "87%"}}}` | Valid JSON | ` [{"name": "fetch_data", "arguments": {"movie": {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "genres": ["Sci-Fi", "Thriller"], "cast": [{"name": "Leonardo DiCaprio", "role": "Dom Cobb"}, {"name": "Joseph Gordon-Levitt", "role": "Arthur"}], "ratings": {"IMDB": 8.8, "Rotten Tomatoes": "87%"}}}}]` | Valid JSON |
| **Query 14** | {'trip': {'destination': 'Tokyo', 'startDate': '2023-04-15', 'endDate': '2023-04-30', 'travelers': [{'name': 'Alice Johnson', 'passport': '123456789'}, {'name': 'Bob Williams', 'passport': '987654321'}], 'itinerary': [{'day': 1, 'activity': 'Visit Tokyo Tower'}, {'day': 2, 'activity': 'Explore Shibuya Crossing'}], 'hotel': 'The Ritz-Carlton', 'roomType': 'Suite'}} | ` {"trip": {"destination": "Tokyo", "startDate": "2023-04-15", "endDate": "2023-04-30", "travelers": [{"name": "Alice Johnson", "passport": "123456789"}, {"name": "Bob Williams", "passport": "987654321"}], "itinerary": [{"day": 1, "activity": "Visit Tokyo Tower"}, {"day": 2, "activity": "Explore Shibuya Crossing"}], "hotel": "The Ritz-Carlton", "roomType": "Suite"}}` | Valid JSON | ` [{"name": "create_trip_plan", "arguments": {"trip": {"destination": "Tokyo", "startDate": "2023-04-15", "endDate": "2023-04-30", "travelers": [{"name": "Alice Johnson", "passport": "123456789"}, {"name": "Bob Williams", "passport": "987654321"}], "itinerary": [{"day": 1, "activity": "Visit Tokyo Tower"}, {"day": 2, "activity": "Explore Shibuya Crossing"}], "hotel": "The Ritz-Carlton", "roomType": "Suite"}}}]` | Valid JSON |
| **Query 15** | {'ecommerce': {'orderID': 987654321, 'customer': {'name': 'Emily Smith', 'email': 'emily@example.com'}, 'items': [{'productID': 12345, 'name': 'Smart TV', 'quantity': 1, 'price': 499.99}, {'productID': 67890, 'name': 'Bluetooth Speaker', 'quantity': 2, 'price': 29.99}], 'total': 559.97, 'status': 'Shipped', 'shipping': {'address': '789 Birch Street', 'city': 'Austin', 'state': 'TX', 'zipcode': 73301'}}} | ` {"ecommerce": {"orderID": 987654321, "customer": {"name": "Emily Smith", "email": "emily@example.com"}, "items": [{"productID": 12345, "name": "Smart TV", "quantity": 1, "price": 499.99}, {"productID": 67890, "name": "Bluetooth Speaker", "quantity": 2, "price": 29.99}], "total": 559.97, "status": "Shipped", "shipping": {"address": "789 Birch Street", "city": "Austin", "state": "TX", "zipcode": 73301}}}` | Valid JSON | ` [{"name": "get_ecommerce_order", "arguments": {"order_id": 987654321}}]` | Valid JSON |
| **Query 16** | {'fitness': {'workoutID': 101, 'type': 'Strength Training', 'exercises': [{'name': 'Bench Press', 'sets': 3, 'reps': 10, 'weight': 135}, {'name': 'Squats', 'sets': 4, 'reps': 12, 'weight': 185}], 'date': '2023-07-10', 'duration': '1 hour'}} | ` {"fitness": {"workoutID": 101, "type": "Strength Training", "exercises": [{"name": "Bench Press", "sets": 3, "reps": 10, "weight": 135}, {"name": "Squats", "sets": 4, "reps": 12, "weight": 185}], "date": "2023-07-10", "duration": "1 hour"}}` | Valid JSON | ` [{"name": "log_workout", "arguments": {"workout_id": 101, "exercise_list": [{"name": "Bench Press", "sets": 3, "reps": 10, "weight": 135}, {"name": "Squats", "sets": 4, "reps": 12, "weight": 185}], "date": "2023-07-10", "duration": "1 hour", "activity_type": "Strength Training"}}]` | Valid JSON |
| **Query 17** | {'concert': {'name': 'Rock Fest 2023', 'location': 'Los Angeles', 'date': '2023-09-20', 'lineup': [{'band': 'The Rolling Stones', 'time': '8:00 PM'}, {'band': 'Queen', 'time': '9:30 PM'}], 'tickets': [{'section': 'VIP', 'price': 500}, {'section': 'General Admission', 'price': 150}], 'available': True} | ` {"concert": {"name": "Rock Fest 2023", "location": "Los Angeles", "date": "2023-09-20", "lineup": [{"band": "The Rolling Stones", "time": "8:00 PM"}, {"band": "Queen", "time": "9:30 PM"}], "tickets": [{"section": "VIP", "price": 500}, {"section": "General Admission", "price": 150}], "available": true}}` | Valid JSON | ` [{"name": "concert", "arguments": {"name": "Rock Fest 2023", "location": "Los Angeles", "date": "2023-09-20", "lineup": [{"band": "The Rolling Stones", "time": "8:00 PM"}, {"band": "Queen", "time": "9:30 PM"}], "tickets": [{"section": "VIP", "price": 500}, {"section": "General Admission", "price": 150}], "available": true}}]` | Valid JSON |
| **Query 18** | {'conference': {'name': 'AI Summit', 'date': '2022-10-15', 'location': 'New York', 'topics': ['AI', 'Machine Learning', 'Deep Learning'], 'speakers': [{'name': 'John Smith', 'topic': 'AI'}, {'name': 'Emily Johnson', 'topic': 'Machine Learning'}] | ` {"conference": {"name": "AI Summit", "date": "2022-10-15", "location": "New York", "topics": ["AI", "Machine Learning", "Deep Learning"], "speakers": [{"name": "John Smith", "topic": "AI"}, {"name": "Emily Johnson", "topic": "Machine Learning"}]}}` | Valid JSON | ` [{"name": "conference", "arguments": {"name": "AI Summit", "date": "2022-10-15", "location": "New York", "topics": ["AI", "Machine Learning", "Deep Learning"], "speakers": [{"name": "John Smith", "topic": "AI"}, {"name": "Emily Johnson", "topic": "Machine Learning"}]}}]` | Valid JSON |
| **Query 19** | {'conference': {'name': 'Tech Expo 2023', 'location': 'San Francisco', 'date': '2023-10-15', 'topics': ['AI', 'Blockchain', 'Cybersecurity'], 'speakers': [{'name': 'John McAfee', 'topic': 'Cybersecurity'}, {'name': 'Vitalik Buterin', 'topic': 'Blockchain'}], 'attendees': 3000}} | ` {"conference": {"name": "Tech Expo 2023", "location": "San Francisco", "date": "2023-10-15", "topics": ["AI", "Blockchain", "Cybersecurity"], "speakers": [{"name": "John McAfee", "topic": "Cybersecurity"}, {"name": "Vitalik Buterin", "topic": "Blockchain"}], "attendees": 3000}}` | Valid JSON | ` [{"name": "conference", "arguments": {"name": "Tech Expo 2023", "location": "San Francisco", "date": "2023-10-15", "topics": ["AI", "Blockchain", "Cybersecurity"], "speakers": [{"name": "John McAfee", "topic": "Cybersecurity"}, {"name": "Vitalik Buterin", "topic": "Blockchain"}], "attendees": 3000}}]` | Valid JSON |
| **Query 20** | {'survey': {'title': 'Customer Satisfaction', 'responses': [{'question': 'How would you rate our service?', 'rating': 5}, {'question': 'What can we improve?', 'text': 'Faster shipping'}], 'summary': {'averageRating': 4.8, 'commonSuggestions': ['Lower prices', 'More products']}}} | ` {"survey": {"title": "Customer Satisfaction", "responses": [{"question": "How would you rate our service?", "rating": 5}, {"question": "What can we improve?", "text": "Faster shipping"}], "summary": {"averageRating": 4.8, "commonSuggestions": ["Lower prices", "More products"]}}}` | Valid JSON | ` [{"name": "analyze_survey", "arguments": {"survey": {"title": "Customer Satisfaction", "responses": [{"question": "How would you rate our service?", "rating": 5}, {"question": "What can we improve?", "text": "Faster shipping"}], "summary": {"averageRating": 4.8, "commonSuggestions": ["Lower prices", "More products"]}}}}]` | Valid JSON |
| **Query 21** | {'socialMedia': {'platform': 'Twitter', 'user': {'username': 'janedoe', 'followers': 5000, 'following': 300, 'tweets': [{'date': '2023-07-10', 'content': 'Just got a new job!'}, {'date': '2023-08-01', 'content': 'Loving my new apartment.'}], 'likes': 100, 'retweets': 10}}} | ` {"socialMedia": {"platform": "Twitter", "user": {"username": "janedoe", "followers": 5000, "following": 300, "tweets": [{"date": "2023-07-10", "content": "Just got a new job!"}, {"date": "2023-08-01", "content": "Loving my new apartment."}], "likes": 100, "retweets": 10}}}` | Valid JSON | ` [{"name": "analyze_social_media_activity", "arguments": {"social_media_data": [{"platform": "Twitter", "user": {"username": "janedoe", "followers": 5000, "following": 300, "tweets": [{"date": "2023-07-10", "content": "Just got a new job!"}, {"date": "2023-08-01", "content": "Loving my new apartment."}], "likes": 100, "retweets": 10}}]}}]` | Valid JSON |
| **Query 22** | {'artExhibit': {'name': 'Modern Art Showcase', 'location': 'Museum of Fine Arts', 'date': '2023-11-05', 'artists': [{'name': 'Pablo Picasso', 'works': ['The Weeping Woman', 'Guernica']}, {'name': 'Vincent van Gogh', 'works': ['Starry Night', 'Sunflowers']}], 'tickets': {'price': 25, 'available': True}}} | ` {"artExhibit": {"name": "Modern Art Showcase", "location": "Museum of Fine Arts", "date": "2023-11-05", "artists": [{"name": "Pablo Picasso", "works": ["The Weeping Woman", "Guernica"]}, {"name": "Vincent van Gogh", "works": ["Starry Night", "Sunflowers"]}], "tickets": {"price": 25, "available": true}}}` | Valid JSON | ` [{"name": "generate_art_exhibit_ticket", "arguments": {"name": "Modern Art Showcase", "location": "Museum of Fine Arts", "date": "2023-11-05", "artists": [{"name": "Pablo Picasso", "works": ["The Weeping Woman", "Guernica"]}, {"name": "Vincent van Gogh", "works": ["Starry Night", "Sunflowers"]}], "tickets": {"price": 25, "available": true}}}]` | Valid JSON |
| **Query 23** | {'banking': {'accountHolder': 'John Doe', 'accountNumber': '12345678', 'transactions': [{'date': '2023-01-01', 'description': 'Deposit', 'amount': 1000}, {'date': '2023-02-15', 'description': 'Withdrawal', 'amount': 500}], 'balance': 500}} | ` {"banking": {"accountHolder": "John Doe", "accountNumber": "12345678", "transactions": [{"date": "2023-01-01", "description": "Deposit", "amount": 1000}, {"date": "2023-02-15", "description": "Withdrawal", "amount": 500}], "balance": 500}}` | Valid JSON | ` [{"name": "get_account_balance", "arguments": {"account_holder": "John Doe", "account_number": "12345678", "transactions": [{"date": "2023-01-01", "description": "Deposit", "amount": 1000}, {"date": "2023-02-15", "description": "Withdrawal", "amount": 500}]}}]` | Valid JSON |
| **Query 24** | {'scientificResearch': {'studyID': 654321, 'title': 'Gene Therapy for Cancer', 'researchers': [{'name': 'Dr. Alice Doe', 'institution': 'Harvard University'}, {'name': 'Dr. Bob Smith', 'institution': 'MIT'}], 'status': 'In Progress', 'funding': 5000000}} | ` {"scientificResearch": {"studyID": 654321, "title": "Gene Therapy for Cancer", "researchers": [{"name": "Dr. Alice Doe", "institution": "Harvard University"}, {"name": "Dr. Bob Smith", "institution": "MIT"}], "status": "In Progress", "funding": 5000000}}` | Valid JSON | ` [{"name": "get_research", "arguments": {"study_id": 654321}}]` | Valid JSON |
| **Query 25** | {'musicConcert': {'concertID': 987123, 'artist': 'The Beatles', 'date': '1965-08-15', 'venue': 'Shea Stadium', 'setlist': [{'song': 'Twist and Shout'}, {'song': 'Help!'}], 'tickets': {'price': 5, 'soldOut': True}}} | ` {"musicConcert": {"concertID": 987123, "artist": "The Beatles", "date": "1965-08-15", "venue": "Shea Stadium", "setlist": [{"song": "Twist and Shout"}, {"song": "Help!"}], "tickets": {"price": 5, "soldOut": true}}}` | Valid JSON | ` [{"name": "music_concert", "arguments": {"concertid": 987123, "artist": "The Beatles", "date": "1965-08-15", "venue": "Shea Stadium", "setlist": [{"song": "Twist and Shout"}, {"song": "Help!"}], "tickets": {"price": 5, "soldout": true}}}]` | Valid JSON |
| **Query 26** | {'fitnessChallenge': {'challengeID': 456789, 'name': '30-Day Fitness Challenge', 'participants': [{'name': 'John Doe', 'progress': 'Week 2'}, {'name': 'Jane Smith', 'progress': 'Week 3'}], 'startDate': '2023-01-01', 'endDate': '2023-01-30'}} | ` {"fitnessChallenge": {"challengeID": 456789, "name": "30-Day Fitness Challenge", "participants": [{"name": "John Doe", "progress": "Week 2"}, {"name": "Jane Smith", "progress": "Week 3"}], "startDate": "2023-01-01", "endDate": "2023-01-30"}}` | Valid JSON | ` [{"name": "get_fitness_challenge", "arguments": {"challengeID": 456789}}]` | Valid JSON |
| **Query 27** | {'stockMarket': {'stock': {'symbol': 'AAPL', 'company': 'Apple Inc.', 'price': 150.50, 'marketCap': '2.5T', 'dividendYield': '0.5%'}, 'sector': 'Technology'}} | ` {"stockMarket": {"stock": {"symbol": "AAPL", "company": "Apple Inc.", "price": 150.5, "marketCap": "2.5T", "dividendYield": "0.5%"}, "sector": "Technology"}}` | Valid JSON | ` [{"name": "stock_market", "arguments": {"stock": {"symbol": "AAPL", "company": "Apple Inc.", "price": 150.5, "marketCap": "2.5T", "dividendYield": "0.5%"}, "sector": "Technology"}}]` | Valid JSON |
| **Query 28** | {'movieReview': {'title': 'The Godfather', 'director': 'Francis Ford Coppola', 'year': 1972, 'ratings': {'IMDB': 9.2, 'Rotten Tomatoes': '98%'}, 'review': 'An absolute classic. A must-watch for any film enthusiast.'}} | ` {"movieReview": {"title": "The Godfather", "director": "Francis Ford Coppola", "year": 1972, "ratings": {"IMDB": 9.2, "Rotten Tomatoes": "98%"}, "review": "An absolute classic. A must-watch for any film enthusiast."}}` | Valid JSON | ` [{"name": "analyze_movie_review", "arguments": {"title": "The Godfather", "director": "Francis Ford Coppola", "year": 1972, "ratings": {"IMDB": 9.2, "Rotten Tomatoes": "98%"}, "review": "An absolute classic. A must-watch for any film enthusiast."}}]` | Valid JSON |
| **Query 29** | {'onlineCourse': {'courseID': 123456, 'title': 'Data Science 101', 'instructor': {'name': 'Dr. Alice Doe', 'institution': 'MIT'}, 'students': 500, 'startDate': '2023-09-01', 'endDate': '2023-12-01'}} | ` {"onlineCourse": {"courseID": 123456, "title": "Data Science 101", "instructor": {"name": "Dr. Alice Doe", "institution": "MIT"}, "students": 500, "startDate": "2023-09-01", "endDate": "2023-12-01"}}` | Valid JSON | ` [{"name": "retrieve_online_course", "arguments": {"course_id": 123456}}]` | Valid JSON |
| **Query 30** | {'travelItinerary': {'itineraryID': 789012, 'destination': 'Paris', 'departureDate': '2023-06-15', 'returnDate': '2023-06-30', 'traveler': {'name': 'John Doe', 'passport': '123456789'}, 'accommodation': {'hotel': 'Le Meurice', 'roomType': 'Deluxe Suite'}}} | ` {"travelItinerary": {"itineraryID": 789012, "destination": "Paris", "departureDate": "2023-06-15", "returnDate": "2023-06-30", "traveler": {"name": "John Doe", "passport": "123456789"}, "accommodation": {"hotel": "Le Meurice", "roomType": "Deluxe Suite"}}}` | Valid JSON | ` [{"name": "generate_travel_plan", "arguments": {"itinerary_id": 789012, "destination": "Paris", "departure_date": "2023-06-15", "return_date": "2023-06-30", "traveler": {"name": "John Doe", "passport": "123456789"}, "accommodation": {"hotel": "Le Meurice", "room_type": "Deluxe Suite"}}}]` | Valid JSON |

**Finetuned Model Valid JSON Responses: 30/30 (100%)**

**Base Model Valid JSON Responses: 30/30 (100%)**